In [ ]:
# from the recent captures directory
# for every folder
# read in the files


In [ ]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import pandas as pd
import os
import sys
import subprocess
import cv2
import shutil

def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath + "dataImport/")
import rwWorkingTSDf
from rwWorkingTSDf import writeWorkingTSDf, readWorkingTSDF, fnString_to_dt, dt_to_fnString
recentCapPath = repoPath + "recentCaptures/"

In [ ]:
def splitVidopencv(input_file_location, boundary_indexes, output_paths):
    cap = cv2.VideoCapture(input_file_location)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'avc1')
    
    for i, (start_frame, end_frame) in enumerate(boundary_indexes):
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        out = cv2.VideoWriter(output_paths[i], fourcc, fps, (width, height))

        for frame_idx in range(start_frame, end_frame):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)

        out.release()
        print(f"Saved: {output_paths[i]}")

    cap.release()

def geBulkLocation(time, responsiblePartyName, instanceName, developingPartyName, deviceName, dataType, dataSource):
    dataFolderName = "_".join([responsiblePartyName, instanceName, developingPartyName, deviceName, dataType, dataSource]) + "/"
    dataFolderName += time.strftime("%Y") + "/" + time.strftime("%m-%d") + "/"
    return repoPath + "bulkData/" + dataFolderName


In [ ]:
foldersToImport = sorted(os.listdir(recentCapPath))
for f in foldersToImport:
    camInfo = f.split("_")[:-2]
    folderPath = recentCapPath + f + "/"
    # get a list of file name bases (withouth the file types)
    fileNameBases = sorted(list(set([x.split(".")[0] for x in os.listdir(folderPath) if x != "new.mp4"])))
    
    # combine all parquets
    for i, baseFileName in enumerate(fileNameBases):
        source = folderPath + baseFileName + ".parquet.gzip"
        rDf = pd.read_parquet(source)
        rDf.index = rDf.index.tz_convert('UTC')
        if i == 0:
            readDf = rDf
        else:
            readDf = pd.concat([readDf, rDf])


    # for each file in a folder
    for baseFileName in fileNameBases:
        startTime = fnString_to_dt(baseFileName.split("_")[-2]).astimezone(ZoneInfo("UTC"))
        endTime = fnString_to_dt(baseFileName.split("_")[-1]).astimezone(ZoneInfo("UTC"))
        # if not a boundary file rename, move and contuinue
        if startTime.day == endTime.day:
            # generate new file name
            newFileName = dt_to_fnString(startTime) + "_" + dt_to_fnString(endTime) + ".mp4"
            source = folderPath + baseFileName + ".mp4"
            destination = geBulkLocation(startTime, *camInfo)
            # move to the right location
            if not os.path.exists(destination):
                print("made directory " + destination)
                os.makedirs(destination)
            shutil.copy2(source, destination  + newFileName)
            continue

        # the file contains more than one day of data
        frameTimes = readDf.copy()
        # print(frameTimes.head().index)
        frameTimes.index = pd.to_datetime(frameTimes.index, utc=True)
        frameTimes['date'] = frameTimes.index.normalize()
        boundaries = frameTimes.groupby('date').apply(lambda x: (x.index[0], x.index[-1]), include_groups=False)
        index_boundaries = [(frameTimes.index.get_loc(start), frameTimes.index.get_loc(end))
                                for day, (start, end) in boundaries.items()]
        
        output_paths = [geBulkLocation(x[0], *camInfo) + 
                        dt_to_fnString(x[0]) + "_" + dt_to_fnString(x[1]) + ".mp4" 
                        for x in boundaries]

        splitVidopencv(folderPath + baseFileName  + ".mp4", index_boundaries, output_paths)
        writeWorkingTSDf(*camInfo, readDf)

    # delete the folder
    #os.rmdir(recentCapPath + f)
